# Information density using sentence semantic similarity
+ normalize text and create sentences
+ create an embedding for every sentence
+ reduce complexity using cluster embeddings
+ NOT YET pick the most central element in order to represent the cluster
+ count clusters

In [1]:
import re
import numpy as np

from pyvis.network import Network
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

2023-03-09 19:37:14.980667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 19:37:16.455370: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-09 19:37:16.455480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-09 19:37:16.455493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia 

In [2]:
import utils

## Texts

In [3]:
# Choosing redundant verses on two versions
# W. Blake - Tiger
# via https://poemario.com/el-tigre/
# via https://www.zendalibros.com/5-poemas-de-william-blake/
raw_text = """
¡Tigre! ¡Tigre!, fuego que ardes
En los bosques de la noche,
¿Qué mano inmortal, qué ojo
Pudo idear tu terrible simetría?

¡Tigre! ¡Tigre! luz llameante
En los bosques de la noche,
¿Qué ojo o mano inmortal
Osó idear tu terrible simetría?
 
Tigre, tigre, que te enciendes en luz
por los bosques de la noche
¿qué mano inmortal, qué ojo
pudo idear tu terrible simetría?

Tigre, tigre, que te enciendes en luz,
por los bosques de la noche
¿qué mano inmortal, qué ojo
osó idear tu terrible simetría?
"""

In [41]:
# split into lines
lines = utils.split_into_lines(raw_text)
len(lines), lines

(16,
 ['¡Tigre! ¡Tigre!, fuego que ardes',
  'En los bosques de la noche,',
  '¿Qué mano inmortal, qué ojo',
  'Pudo idear tu terrible simetría?',
  '¡Tigre! ¡Tigre! luz llameante',
  'En los bosques de la noche,',
  '¿Qué ojo o mano inmortal',
  'Osó idear tu terrible simetría?',
  'Tigre, tigre, que te enciendes en luz',
  'por los bosques de la noche',
  '¿qué mano inmortal, qué ojo',
  'pudo idear tu terrible simetría?',
  'Tigre, tigre, que te enciendes en luz,',
  'por los bosques de la noche',
  '¿qué mano inmortal, qué ojo',
  'osó idear tu terrible simetría?'])

In [58]:
embeddings = [sentence_model.encode(_) for _ in lines]
emb_arr = np.array(embeddings)
emb_arr.shape

(16, 768)

In [63]:
from sklearn.cluster import KMeans

# kmeans = KMeans(max_iter=1000, random_state=True, n_init=50).fit(X=emb_arr.astype('double'))
clusterer = KMeans(n_clusters=4, max_iter=1000, random_state=True, n_init=50) \
    .fit(X=emb_arr.astype('double'))
clusterer.labels_.max() + 1, clusterer.labels_

(4, array([3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1], dtype=int32))

In [64]:
mapping = [(label, lines[index]) for index, label in enumerate(clusterer.labels_)]
# dd.sort()
sorted(mapping)

[(0, 'En los bosques de la noche,'),
 (0, 'En los bosques de la noche,'),
 (0, 'por los bosques de la noche'),
 (0, 'por los bosques de la noche'),
 (1, 'Osó idear tu terrible simetría?'),
 (1, 'Pudo idear tu terrible simetría?'),
 (1, 'osó idear tu terrible simetría?'),
 (1, 'pudo idear tu terrible simetría?'),
 (2, '¿Qué mano inmortal, qué ojo'),
 (2, '¿Qué ojo o mano inmortal'),
 (2, '¿qué mano inmortal, qué ojo'),
 (2, '¿qué mano inmortal, qué ojo'),
 (3, 'Tigre, tigre, que te enciendes en luz'),
 (3, 'Tigre, tigre, que te enciendes en luz,'),
 (3, '¡Tigre! ¡Tigre! luz llameante'),
 (3, '¡Tigre! ¡Tigre!, fuego que ardes')]

In [65]:
clusterer = KMeans(max_iter=1000, random_state=True, n_init=50) \
    .fit(X=emb_arr.astype('double'))
clusterer.labels_.max() + 1, clusterer.labels_

(8, array([4, 1, 0, 2, 5, 1, 6, 7, 3, 1, 0, 2, 3, 1, 0, 7], dtype=int32))

In [66]:
mapping = [(label, lines[index]) for index, label in enumerate(clusterer.labels_)]
# dd.sort()
sorted(mapping)

[(0, '¿Qué mano inmortal, qué ojo'),
 (0, '¿qué mano inmortal, qué ojo'),
 (0, '¿qué mano inmortal, qué ojo'),
 (1, 'En los bosques de la noche,'),
 (1, 'En los bosques de la noche,'),
 (1, 'por los bosques de la noche'),
 (1, 'por los bosques de la noche'),
 (2, 'Pudo idear tu terrible simetría?'),
 (2, 'pudo idear tu terrible simetría?'),
 (3, 'Tigre, tigre, que te enciendes en luz'),
 (3, 'Tigre, tigre, que te enciendes en luz,'),
 (4, '¡Tigre! ¡Tigre!, fuego que ardes'),
 (5, '¡Tigre! ¡Tigre! luz llameante'),
 (6, '¿Qué ojo o mano inmortal'),
 (7, 'Osó idear tu terrible simetría?'),
 (7, 'osó idear tu terrible simetría?')]